<a href="https://colab.research.google.com/github/rasik-nep/Chatbot-Prototype/blob/main/transaltion_failed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install required packages
!pip install -q transformers peft  accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain sentence-transformers gradio pypdf deep-translator

In [2]:
# import dependencies
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

import os
import gradio as gr
from google.colab import drive

import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [3]:
# specify model huggingface mode name
model_name = "anakin87/zephyr-7b-alpha-sharded"

# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

In [4]:
# fucntion for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [ ]:
# load model
model = load_quantized_model(model_name)

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)

# specify stop token ids
stop_token_ids = [0]

In [6]:
# mount google drive and specify folder path
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/my_documents/'

Mounted at /content/drive


In [7]:
# load pdf files
loader = PyPDFDirectoryLoader(folder_path)
documents = loader.load()
print(documents)
# split the documents in small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) #Chage the chunk_size and chunk_overlap as needed
all_splits = text_splitter.split_documents(documents)

# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)

#embed document chunks
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

# specify the retriever
retriever = vectordb.as_retriever()

[Document(page_content=" \n(1) \n The Constitution of Nepal  \nDate of Publication in Nepal Gazette  \n 20 September 2015 (2072.6.3)  \nThe Constitution of Nepal (First Amendment), 2016  (2072)  \n 28 February 2016 (2072.11.16)  \n \nPreamble : \nWe, the Sovere ign People of Nepal,  \nInternalizing the people's sovereign right and right to autonomy \nand self -rule, while maintaining freedom, sovereignty, territorial \nintegrity, national unity, independence and dignity of Nepal,  \nRecalling the glorious history of historic people's movements, \narmed conflict, dedication and sacrifice undertaken by the \nNepalese people at times for the interest of the nation, \ndemocracy and progressive changes , and respecting for the \nmartyrs and disappeared and victi m citizens,  \nEnding all forms of discrimination and oppression created by the \nfeudalistic, autocratic, centralized, unitary system of \ngovernance,  \nProtecting and  promoting social and cultural solidarity, tolerance \nand har

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# build huggingface pipeline for using zephyr-7b-alpha
pipeline = pipeline(
        "text-generation",
        use_cache=True,
        max_length=2048,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        # device=0
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)

# build conversational retrieval chain with memory (rag) using langchain
def create_conversation(query: str, chat_history: list) -> tuple:
    try:
        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            memory=memory,
            get_chat_history=lambda h: h,
        )

        result = qa_chain({'question': query, 'chat_history': chat_history})
        chat_history.append((query, result['answer']))
        return '', chat_history

    except Exception as e:
        chat_history.append((query, e))
        return '', chat_history

In [13]:
# Build Gradio UI
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label='Chat with your data (Zephyr 7B Alpha)')
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot])

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f9ddb0245e8964c830.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [16]:
# build gradio ui
with gr.Blocks() as demo:

    chatbot = gr.Chatbot(label='Chat with your data (Zephyr 7B Alpha)')
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot])

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1b2b3a4721329bb9fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
from deep_translator import GoogleTranslator
def translate_to_nepali(text_to_translate):
    """
    Translates English text to Nepali using Google Translator.

    Args:
        text_to_translate (str): The input text in English.

    Returns:
        str: The translated text in Nepali.
    """
    try:
        translated_text = GoogleTranslator(source='auto', target='ne').translate(text_to_translate)
        return translated_text
    except Exception as e:
        return f"Translation error: {str(e)}"

# Example usage
english_text = "I want to translate this text"
nepali_translation = translate_to_nepali(english_text)
print("Translated text:", nepali_translation)

Translated text: म यो पाठ अनुवाद गर्न चाहन्छु


In [11]:
from deep_translator import GoogleTranslator

def translate_to_english(nepali_text):
    """
    Translates Nepali text to English using Google Translate.
    Args:
        nepali_text (str): The input text in Nepali.
    Returns:
        str: Translated text in English.
    """
    try:
        # Initialize the Google Translator
        translator = GoogleTranslator(source='ne', target='en')

        # Translate the Nepali text to English
        english_text = translator.translate(nepali_text)

        return english_text
    except Exception as e:
        # Handle any exceptions (e.g., network issues, invalid input)
        return f"Translation error: {str(e)}"

# Example usage
nepali_prompt = "नमस्कार, कस्तो छ?"
english_prompt = translate_to_english(nepali_prompt)
print(f"Translated prompt (English): {english_prompt}")


Translated prompt (English): Hello, how are you?


In [14]:
with gr.Blocks() as demo:
    # Receive Nepali prompts
    nepali_prompt = gr.Textbox(label="Enter your Nepali prompt:")

    # Translate Nepali prompt to English (you'll need to implement this step)
    english_prompt = translate_to_english(nepali_prompt)

    # Pass the translated prompt to create_conversation
    chatbot = gr.Chatbot(label='Chat with your data (Zephyr 7B Alpha)')

    chatbot.submit(create_conversation, [english_prompt], [chatbot])

demo.launch()


AttributeError: 'Chatbot' object has no attribute 'submit'

In [18]:
# build gradio ui
with gr.Blocks() as demo:

    chatbot = gr.Chatbot(label='Chat with your data (Zephyr 7B Alpha)')
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot])

demo.launch()

AttributeError: 'str' object has no attribute '_id'